In [10]:
import pandas as pd
import re
import os

def clean_name(name):
    # Lista de nomes compostos comuns para verificação
    compound_first_names = ["MARIA JOSÉ", "LUIZA HELENA", "MARIA DE LOURDES", "MARIA DAS DORES", "MARIA ELMA"]
    # Checa se o nome é nulo ou vazio
    if pd.isna(name) or name.strip() == '':
        return None
    # Remove caracteres especiais e espaços desnecessários
    clean_name = re.sub(r'[^\w\s]', '', name).strip()
    # Divide o nome em palavras e converte para maiúsculas
    words = clean_name.upper().split()
    # Verifica se as duas primeiras palavras formam um nome composto conhecido
    if len(words) >= 2 and " ".join(words[:2]) in compound_first_names:
        return " ".join(words[:2]).title()
    # Retorna o primeiro nome ou None se não houver palavras
    return words[0].title() if words else None

def clean_phone(phone):
    # Verifica se o telefone é nulo
    if pd.isna(phone):
        return None
    # Remove aspas simples e caracteres não-dígitos, exceto o '+' inicial
    phone = re.sub(r"(?<!^)[^\d]", "", phone.strip("'"))
    # Adiciona o código do país se começar com '+'
    if phone.startswith('+'):
        phone = '55' + phone[3:]
    # Formata o telefone para o padrão brasileiro
    if len(phone) == 13 and phone.startswith('55') and phone[4] == '9':
        return phone[:4] + phone[5:]
    # Verifica se o telefone tem o comprimento correto e começa com '55'
    return phone if len(phone) == 12 and phone.startswith('55') else None

def process_file(input_path):
    # Verifica se o caminho do arquivo é válido
    if not os.path.isfile(input_path):
        print("O caminho especificado não é um arquivo válido.")
        return

    # Carrega a planilha do Excel
    df = pd.read_excel(input_path)

    # Solicita ao usuário os nomes das colunas de nome e telefone
    name_column = input("Digite o nome da coluna que contém os Nomes: ")
    phone_column = input("Digite o nome da coluna que contém os Telefones: ")

    # Verifica se as colunas necessárias estão presentes
    missing_columns = [col for col in [name_column, phone_column] if col not in df.columns]
    if missing_columns:
        print(f"As colunas faltantes são: {', '.join(missing_columns)}")
        return

    # Processa o DataFrame limpando nomes e telefones
    df['Nome Limpo'] = df[name_column].apply(clean_name)
    df['Telefone Limpo'] = df[phone_column].apply(clean_phone)

    # Seleciona e renomeia as colunas para visualização
    df_final = df[['Telefone Limpo', 'Nome Limpo']].dropna()
    df_final.columns = ['Número', 'Primeiro Nome']

    # Exibe um preview dos dados processados
    print("Preview das primeiras 5 linhas:")
    print(df_final.head())
    print("\nPreview das últimas 5 linhas:")
    print(df_final.tail())

    # Solicita confirmação para salvar a planilha processada
    confirm = input("Deseja salvar a planilha? (sim/não): ")
    if confirm.lower() == 'sim':
        filename = input("Digite o nome do arquivo para salvar: ")
        if not filename.endswith('.xlsx'):
            filename += '.xlsx'
        save_path = input("Digite o diretório onde deseja salvar o arquivo: ")
        if not os.path.isdir(save_path):
            print("O diretório especificado não existe.")
        else:
            output_path = os.path.join(save_path, filename)
            df_final.to_excel(output_path, index=False)
            print(f"Planilha salva com sucesso em: {output_path}")
    else:
        print("Operação cancelada.")

def main():
    # Loop principal para processar múltiplas planilhas
    while True:
        input_path = input("Digite o caminho completo da planilha a ser tratada (incluindo o nome do arquivo): ")
        process_file(input_path)
        another = input("Deseja processar outra planilha? (sim/não): ")
        if another.lower() != 'sim':
            break

if __name__ == "__main__":
    main()

input("Pressione Enter para encerrar o programa...")


Digite o caminho completo da planilha a ser tratada (incluindo o nome do arquivo):  s


O caminho especificado não é um arquivo válido.


Deseja processar outra planilha? (sim/não):  nao
Pressione Enter para encerrar o programa... 


''